## Imports

In [85]:
import sys 
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline  

from sklearn import svm, linear_model
from sklearn.linear_model import SGDClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib

import torch
import torchvision 
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset, DataLoader

sys.path.insert(0, '../../../Utils/')
sys.path.insert(0, '../../../')
import cyphercat as cc

import models
from train import *
from metrics import * 
from SVC_Utils import *

#audio
import librosa as libr

print("Python: %s" % sys.version)
print("Pytorch: %s" % torch.__version__)

# determine device to run network on (runs on gpu if available)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


Python: 3.6.5 (default, Jul  6 2018, 19:12:46) 
[GCC 5.4.0 20160609]
Pytorch: 0.4.0


## NN Hyperparameters

In [96]:
batch_size = 128
lr = 0.001
k = 3

pretrained = True #run this with networks that have already been trained

transform_type = 'SFTF' #either STFT or MFCC  

data = 'Libri' #'Libri' or 'VOiCES'
defense = 'dimen_reduc_top3_breakpost'

In [87]:
### Speech preprocessing

class tensorToMFCC:
    def __call__(self, y):
#         y = y.numpy()
        dims = y.shape
        y = libr.feature.melspectrogram(np.reshape(y, (dims[1],)), 16000, n_mels=number_of_mels,
                               fmax=8000)
        y = libr.feature.mfcc(S = libr.power_to_db(y))
        y = torch.from_numpy(y)                           
        return y.float()

class STFT:
    def __call__(self,y):
        dims = y.shape
        y = np.abs(libr.core.stft(np.reshape(y, (dims[1],))))
        y = torch.from_numpy(y).permute(1,0)
        return y.float()

if transform_type == 'SFTF':
    target_net_type = cc.ft_cnn_classifer
    shadow_net_type = cc.ft_cnn_classifer
    in_size = 94# 20 forMFCC,  94 for STFT
    transform  = STFT() ## STFT or MFCC
elif transform_type == 'MFCC':
    transform  = tensorToMFCC()
    target_net_type = cc.MFCC_cnn_classifier
    shadow_net_type = cc.MFCC_cnn_classifier
    in_size = 20

## Audio hyperparameters

In [88]:
n_seconds = 3
n_epochs = 50
shadow_epochs = 15
n_epochs_attack = 100
sampling_rate = 16000
number_of_mels =128
lr = 0.001

# attacking means data for a target & shadow network.
# This will also split "out data" from totally different speakers -- data none of the 
# other networks have seen, for training & testing the attack network. This will be
# an equivalent amount of data to the train split as defined about

%load_ext autoreload
%autoreload 2
sys.path.insert(0, './../../../Utils')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load audio data: VOiCES or LibriSpeech, & Split into valid sequences

In [89]:
print('Loading splits')
subset = 'room-1'
if data == 'Libri':
    [dfs, sample_df] = cc.Libri_preload_and_split()
    print('Initializing dataset')
    valid_sequence_train_target = cc.LibriSpeechDataset(df=dfs[0], transform = transform)
    valid_sequence_test_target = cc.LibriSpeechDataset(df=dfs[1], transform = transform)
    valid_sequence_train_shadow = cc.LibriSpeechDataset(df=dfs[2], transform = transform)
    valid_sequence_test_shadow = cc.LibriSpeechDataset(df=dfs[3], transform = transform)
    valid_sequence_attack_in = cc.LibriSpeechDataset(df=dfs[4], transform = transform)
    valid_sequence_attack_out = cc.LibriSpeechDataset(df=dfs[5], transform = transform)

    print('Succesfully loaded libri-speech')
elif data == 'VOiCES':
    [dfs, sample_df] = cc.Voices_preload_and_split(subset = subset)
    print('Initializing dataset')
    valid_sequence_train_target = cc.Voices_dataset(df=dfs[0], transform = transform)
    valid_sequence_test_target = cc.Voices_dataset(df=dfs[1], transform = transform)
    valid_sequence_train_shadow = cc.Voices_dataset(df=dfs[2], transform = transform)
    valid_sequence_test_shadow = cc.Voices_dataset(df=dfs[3], transform = transform)
    valid_sequence_attack_in = cc.Voices_dataset(df=dfs[4], transform = transform)
    valid_sequence_attack_out = cc.Voices_dataset(df=dfs[5], transform = transform)

Loading splits
Initialising LibriSpeechDataset with minimum length = 3s and subset = train-clean-100
	 Finished indexing train-clean-100. 27949 usable files found.
	 Finished indexing test-clean. 2225 usable files found.
Build/load speaker membership inference splits
Found default speaker splits, loading dataframe
Build/load sample membership inference splits
Found default sample splits, loading dataframe

 ------- Speaker split statistics ------- 
		 ---- Split 0 ---- 
	Unique speakers 	 Samples
Male:		 63		 5539
Female:		 62		 5573
Total:		 125		 11112
		 ---- Split 1 ---- 
	Unique speakers 	 Samples
Male:		 63		 1414
Female:		 62		 1427
Total:		 125		 2841
		 ---- Split 2 ---- 
	Unique speakers 	 Samples
Male:		 31		 3519
Female:		 31		 3478
Total:		 62		 6997
		 ---- Split 3 ---- 
	Unique speakers 	 Samples
Male:		 32		 3462
Female:		 32		 3537
Total:		 64		 6999
		 ---- Split 4 ---- 
	Unique speakers 	 Samples
Male:		 16		 1414
Female:		 16		 1427
Total:		 32		 2841
		 ---- Split 

In [90]:
# # # to look at the index file:

# # #look at splits file for reference
# dff = pd.read_csv(os.getcwd()+'/../../../Datasets/splits/libri-train-clean-100/libri_4.csv')
# print(dff.head())
# dff2 = pd.read_csv(os.getcwd()+'/../../../Datasets/splits/VOiCES-room-1/VOiCES_0.csv')
# print(dff2.head())


# # df = pd.read_csv(os.getcwd()+'/../../../Datasets/VOiCES-room-1.index.csv')
# # df.head()

# # g = df.groupby(['id','Section']).groups

# # dfn = pd.DataFrame(columns = ['id','Section'])
# # idx = 0
# # for key in g.keys():
# #     dfn.at[idx,'id']=key[0]
# #     dfn.at[idx,'Section']=key[1]
# #     idx +=1

# # print(len(np.unique(df.id)), 'speakers')
# # print(len(df), 'files')
# # print(dfn.groupby('id').count().min()[0], 'tracks min')
# # print(dfn.groupby('id').count().max()[0], 'tracks max')
# # print(dfn.groupby('id').count().mean()[0], 'tracks mean')

# # print('min speaker minutes',df.groupby('id').mean()['speaker_minutes'].min())
# # print('max speaker minutes',df.groupby('id').mean()['speaker_minutes'].max())
# # print('mean speaker minutes',df.groupby('id').mean()['speaker_minutes'].mean())

In [91]:
# splits = [.2,.8]
# df1 = pd.DataFrame(columns = df.columns)
# df2 = pd.DataFrame(columns = df.columns)
# # For each speaker, identify unique segments: 
# for spkr_id in df.id.unique():
#     mini_df = df[df['id'] == spkr_id]
#     # Identify segments:
#     n_seg = len(mini_df.Section.unique())
#     seg1 = round(splits[0]*n_seg)
#     # Segments are not ordered in a particular way, so just pick the first few for seg1
#     seg1s = mini_df.Section.unique()[:seg1]
#     df1 = df1.append(mini_df[mini_df['Section'].isin(seg1s)])
#     df2 = df2.append(mini_df[~mini_df['Section'].isin(seg1s)])

In [92]:
batch_size = 32


# Loaders for data for target model & shadow model 
target_train_loader = DataLoader(valid_sequence_train_target,
                      batch_size=batch_size,
                      shuffle=True,
                      num_workers=8,
                    drop_last = True
                     # pin_memory=True # CUDA only
                     )


target_test_loader = DataLoader(valid_sequence_test_target,
                      batch_size=batch_size,
                      shuffle=True,
                      num_workers=8
                     # pin_memory=True # CUDA only
                     )

shadow_train_loader = DataLoader(valid_sequence_train_shadow,
                      batch_size=batch_size,
                      shuffle=True,
                      num_workers=8,
                    drop_last = True
                     # pin_memory=True # CUDA only
                     )


shadow_test_loader = DataLoader(valid_sequence_test_shadow,
                      batch_size=batch_size,
                      shuffle=True,
                      num_workers=8
                     # pin_memory=True # CUDA only
                     )


test_loader_in = DataLoader(valid_sequence_attack_in,
                      batch_size=batch_size,
                      shuffle=True,
                      num_workers=8
                     # pin_memory=True # CUDA only
                     )


test_loader_out = DataLoader(valid_sequence_attack_out,
                      batch_size=batch_size,
                      shuffle=True,
                      num_workers=8
                     # pin_memory=True # CUDA only
                     )


## Set up model tracking

In [93]:
#table with summary

summary_file = 'summary.pkl'
columns = ['Data','Transform','Training epochs', '# speakers','Train accuracy', 'Test accuracy', 
           'Attack type', 'Defense','Attack Accuracy', 'Attack Precision','Attack Recall', 'Thresholds']


df = pd.DataFrame(columns = columns)
    
df_idx = len(df)

#set a bunch of known values
df.at[df_idx,'Transform'] =transform_type
df.at[df_idx,'Training epochs'] = n_epochs
df.at[df_idx,'Attack type'] = 1
df.at[df_idx,'Defense'] = defense
df.at[df_idx,'Data'] = data

# Initialize/Train Targets
The model being attacked; if network, architecture can differ from that of shadow network.

In [110]:
#Initialize NN

#in_size defined above
n_hidden = 512
n_classes = valid_sequence_test_target.num_speakers
print(n_classes,' speakers')
df.at[df_idx,'# speakers']=n_classes


target_net = target_net_type(n_classes).to(device)
target_net.apply(models.weights_init)

target_loss = nn.CrossEntropyLoss()
target_optim = optim.Adam(target_net.parameters(), lr=lr)

125  speakers


In [111]:
#file name for this set of hyperparameters
fn = 'model_weights/CNN_voice_classifier'+data+'_target_'+transform_type+str(n_epochs-1)

#Train NN
if not pretrained:
    train_accuracy, test_accuracy = cc.train(target_net, target_train_loader, target_test_loader, target_optim, target_loss, n_epochs, verbose = False) 
    df.at[df_idx,'Train accuracy'] =round(train_accuracy,4)/100
    df.at[df_idx,'Test accuracy'] = round(test_accuracy,4)/100
    cc.save_checkpoint(model = target_net, optimizer = target_optim,
                           epoch = n_epochs-1, data_descriptor = data, 
                           accuracy = [train_accuracy, test_accuracy],
                           filename = fn)
    
else:
    fn = fn + '.pth.tar'
    print(fn)
    cc.load_checkpoint(model = target_net, checkpoint = fn)


model_weights/CNN_voice_classifierLibri_target_SFTF49.pth.tar
Succesfully loaded checkpoint 
Dataset: Libri 
Epoch: 14 
Loss: None           
Accuracy: [99.15345821325649, 94.68497008095741]


# Initialize/Train Shadow Model
Shadow model mimics the target network, emulating the target model's differences in prediction probabilities for samples in and out of its dataset. For this attack, only one shadow model is used. 

In [112]:
valid_sequence_train_shadow.num_speakers

62

In [113]:
# IF defending:
if defense == 'dimen_reduc_top3_breakpost':
    print(defense)
    target_net = cc.defenses.dimensionality_reduction(model = target_net, n_top = 3, break_posterior = True)
    

dimen_reduc_top3_breakpost


In [114]:
#Initialize models

n_classes = valid_sequence_test_shadow.num_speakers
print('n shadow speakers',n_classes)

#NN
shadow_net = shadow_net_type(n_classes).to(device)
shadow_net.apply(models.weights_init)

shadow_loss = nn.CrossEntropyLoss()
shadow_optim = optim.Adam(shadow_net.parameters(), lr=lr)

n shadow speakers 64


# Initialize Attack Model
A binary classifier to determine membership. 

In [117]:
# Attack the network: 

attack_net_nn = models.mlleaks_mlp(n_in=k).to(device)
attack_loss = nn.BCEWithLogitsLoss()
attack_optim_nn= optim.Adam(attack_net_nn.parameters(), lr=lr)

df_pr = cc.ml_leaks1(target=target_net, shadow_model = shadow_net, attacker_model = attack_net_nn,
            target_in_loader = test_loader_in, target_out_loader = test_loader_out,
            shadow_train_loader = shadow_train_loader, shadow_out_loader=shadow_test_loader,
            shadow_optim = shadow_optim, attack_optim = attack_optim_nn, 
            shadow_criterion = shadow_loss, attack_criterion = attack_loss, 
            shadow_epochs = shadow_epochs, attack_epochs = n_epochs_attack, retrain = True)

---- Training shadow network ----
[0/15]
Training:

Accuracy = 88.26 %%


Test:

Accuracy = 2.09 %%


[1/15]
Training:

Accuracy = 93.42 %%


Test:

Accuracy = 2.39 %%


[2/15]
Training:

Accuracy = 95.56 %%


Test:

Accuracy = 2.41 %%


[3/15]
Training:

Accuracy = 95.50 %%


Test:

Accuracy = 1.74 %%


[4/15]
Training:

Accuracy = 95.60 %%


Test:

Accuracy = 1.01 %%


[5/15]
Training:

Accuracy = 97.06 %%


Test:

Accuracy = 1.94 %%


[6/15]
Training:

Accuracy = 97.35 %%


Test:

Accuracy = 1.43 %%


[7/15]
Training:

Accuracy = 96.86 %%


Test:

Accuracy = 2.17 %%


[8/15]
Training:

Accuracy = 98.04 %%


Test:

Accuracy = 2.14 %%


[9/15]
Training:

Accuracy = 97.58 %%


Test:

Accuracy = 2.77 %%


[10/15]
Training:

Accuracy = 97.16 %%


Test:

Accuracy = 2.01 %%


[11/15]
Training:

Accuracy = 96.83 %%


Test:

Accuracy = 1.37 %%


[12/15]
Training:

Accuracy = 98.48 %%


Test:

Accuracy = 2.51 %%


[13/15]
Training:

Accuracy = 98.08 %%


Test:

Accuracy = 1.46 %%


[14/15]
Tr

Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7fee20931978>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 349, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 328, in _shutdown_workers
    self.worker_result_queue.get()
  File "/usr/local/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/usr/local/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 70, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/local/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/usr/local/lib/python3.6/multiprocessing/resource_sharer.py", line 87, in get_connection
    c = Client(address, authkey=process.current_process

---- Evaluate attack ----


In [118]:
df_pr

,Thresholds,Accuracy,Precision,Recall
0,0.500,49.832027,0.0,0.0
1,0.505,49.832027,0.0,0.0
2,0.510,49.832027,0.0,0.0
3,0.515,49.832027,0.0,0.0
4,0.520,49.832027,0.0,0.0
5,0.525,49.832027,0.0,0.0
6,0.530,49.832027,0.0,0.0
7,0.535,49.832027,0.0,0.0
8,0.540,49.832027,0.0,0.0
9,0.545,49.832027,0.0,0.0


In [120]:
# Ascertain best results

df.at[df_idx,'Attack Precision'] = round(df_pr[df_pr['Accuracy']==df_pr['Accuracy'].max()].Precision.values[0],4)
df.at[df_idx,'Attack Recall'] = round(df_pr[df_pr['Accuracy']==df_pr['Accuracy'].max()].Recall.values[0],4)
df.at[df_idx,'Attack Accuracy'] = round(df_pr[df_pr['Accuracy']==df_pr['Accuracy'].max()].Accuracy.values[0],4)/100
df.at[df_idx,'Threshold'] = round(df_pr[df_pr['Accuracy']==df_pr['Accuracy'].max()].Thresholds.values[0],4)

In [133]:
# Add results to table

try:
    df_summ = pd.read_pickle(summary_file)
    print('Loading summary file')

except:
    print('Creating new summary file')

# Append row you created
df_summ = df_summ.append(df) 



In [ ]:
# Save updated df
df_summ.to_pickle(summary_file)

# Evaluate Attack Nets
How well the trained attack models classify a sample as in or out of a target model's training dataset, and how performance is affected by target hyperparameters and which models attack which targets.

In [60]:
df['# speakers'] =df['# speakers'].astype(float)
df['Training epochs'] =df['Training epochs'].astype(float)
df['Attack type'] =df['Attack type'].astype(float)

#style table
import seaborn as sns

cg = sns.light_palette("green", as_cmap=True)
cm = sns.light_palette("magenta", as_cmap=True)
bl = sns.light_palette("blue", as_cmap=True)
orr = sns.light_palette("orange", as_cmap=True)
gr = sns.light_palette("gray", as_cmap=True)

# df.style.bar(subset=['Train accuracy', 'Test accuracy'], align='mid', color=['#d65f5f', '#5fba7d'])
s = df.style.\
    format({"Attack Precision": "{:.2%}","Attack Recall": "{:.2%}","Attack Accuracy": "{:.2%}"}).\
    format({"Train accuracy": "{:.2%}","Test accuracy": "{:.2%}"}).\
    hide_index().\

    set_properties(**{'font-size': "16pt",'column-size':"24pt",'width': '100px'})

s

Transform,Training epochs,# speakers,Train accuracy,Test accuracy,Attack type,Attack Precision,Attack Recall,Attack Accuracy,Defense,Data
SFTF,50,125,99.52%,94.97%,1,85.85%,90.76%,87.86%,None,Libri


In [ ]:
    background_gradient(cmap=cg,subset=['Train accuracy', 'Test accuracy']).\
    background_gradient(cmap=bl,subset=['Precision', 'Recall']).\
    background_gradient(cmap=orr,subset=['Training epochs']).\
    background_gradient(cmap=gr,subset=['Attack type']).\
    background_gradient(cmap=cm,subset=['# speakers']).\